In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import Sequential,activations, Input, Model, models
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.layers import AveragePooling2D, MaxPooling2D, Dropout
from tensorflow.keras.layers import LeakyReLU
from distutils.dir_util import copy_tree

import warnings
warnings.simplefilter('ignore')
from tensorflow.keras.layers import Dense,Activation, Dropout, Conv2D, MaxPool2D, Flatten,GlobalAveragePooling2D, GlobalMaxPooling2D, BatchNormalization

In [ ]:
t=copy_tree("../input/happy-whale-and-dolphin/train_images/","/kaggle/temp/train/")

In [ ]:
from PIL import Image
import os, sys
import glob

root_dir = "/kaggle/temp/train/"
for filename in glob.iglob(root_dir + '**/*.jpg', recursive=True):
    im = Image.open(filename)
    imResize = im.resize((128,128), Image.ANTIALIAS)
    imResize.save(filename , 'JPEG')

In [ ]:
train=pd.read_csv('../input/happy-whale-and-dolphin/train.csv', header='infer')
train.head()

In [ ]:
label=train["individual_id"].unique().tolist()
#label.append("new_individual")
label=np.array(label)
label.shape

In [ ]:
#make validation directory
if not os.path.exists("/kaggle/temp/valid/"):
    os.mkdir("/kaggle/temp/valid/")

In [ ]:
for i in label:
    if not os.path.exists("/kaggle/temp/train/"+i):
        os.mkdir("/kaggle/temp/train/"+i)
    if not os.path.exists("/kaggle/temp/valid/"+i):
        os.mkdir("/kaggle/temp/valid/"+i)

In [ ]:
for i in range(len(train["image"])):
    img=train["image"][i]
    t= np.random.random()
    
    if t<= 0.5:
        new_path= os.path.join("/kaggle/temp/train/",train["individual_id"][i],img)
        os.replace('/kaggle/temp/train/'+img,new_path)
    else:
        new_path= os.path.join("/kaggle/temp/valid/",train["individual_id"][i],img)
        os.replace('/kaggle/temp/train/'+img,new_path)

In [ ]:
#apply the image generator on train images for augmention
train_datagen = ImageDataGenerator(featurewise_center= False,
                                  samplewise_center= False,
                                  featurewise_std_normalization=False,
                                  samplewise_std_normalization=False,
                                  zca_whitening=False,
                                  rotation_range=10,
                                  zoom_range=0.1,
                                  width_shift_range=0.1,
                                  height_shift_range=0.1,
                                  horizontal_flip=False,
                                  vertical_flip=False,
                                  rescale=1./255
                                  )

In [ ]:
train_gen = train_datagen.flow_from_directory(directory="/kaggle/temp/train/", target_size=(128,128),batch_size=64)
validation_datagen = ImageDataGenerator(rescale=1./255)
valid_gen = validation_datagen.flow_from_directory(directory='/kaggle/temp/valid/',target_size=(128,128),batch_size=64)

In [ ]:
model=Sequential()
#Convolution layer 1
model.add(Conv2D(filters=32, kernel_size=(5,5), strides=(1,1), padding='valid',input_shape=(128,128, 3)))
model.add(BatchNormalization())
model.add(Activation(LeakyReLU(alpha=0.01)))

model.add(AveragePooling2D((3, 3)))
model.add(Conv2D(filters=64, kernel_size=(4,4), strides=(1,1), padding='valid'))
model.add(BatchNormalization())
model.add(Activation(LeakyReLU(alpha=0.01)))
model.add(AveragePooling2D((3, 3)))
#Convolution layer 2
model.add(Conv2D(filters=96, kernel_size=(3,3), strides=(1,1), padding='valid'))
model.add(BatchNormalization())
model.add(Activation(LeakyReLU(alpha=0.01)))
model.add(AveragePooling2D((3, 3)))


model.add(Conv2D(filters=128, kernel_size=(2,2), strides=(1,1), padding='valid'))
model.add(BatchNormalization())
model.add(Activation(LeakyReLU(alpha=0.01)))

model.add(Conv2D(filters=15587, kernel_size=(2,2), strides=(1,1), padding='valid'))
model.add(Activation('softmax'))
model.add(Flatten())

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

In [ ]:
#setup callback
reduceLROnPlateau = ReduceLROnPlateau(monitor='val_acc',patience=3,verbose=1,factor=0.5,min_lr=0.001)

In [ ]:
model.fit(train_gen,epochs=100,callbacks=[reduceLROnPlateau],validation_data=valid_gen,steps_per_epoch=train_gen.n//train_gen.batch_size,
         validation_steps= valid_gen.n//valid_gen.batch_size,workers=8,use_multiprocessing=True)

In [ ]:
p=copy_tree("../input/happy-whale-and-dolphin/test_images","/kaggle/temp/test/test_img/")

In [ ]:
from PIL import Image
import os, sys
import glob



root_dir = "/kaggle/temp/test/test_img/"


for filename in glob.iglob(root_dir + '**/*.jpg', recursive=True):
    im = Image.open(filename)
    imResize = im.resize((128,128), Image.ANTIALIAS)
    imResize.save(filename , 'JPEG', quality=90)
    

In [ ]:
#apply image genetory on test images
test_datagen = ImageDataGenerator(rescale=1./255)
test_gen = test_datagen.flow_from_directory(directory='/kaggle/temp/test/',target_size=(128,128),batch_size=64
                                        ,class_mode=None,shuffle=False)

In [ ]:
#prediction
predictions= model.predict(test_gen)
predictions.shape

In [ ]:
prediction_prob=[]

for i in range(len(predictions)):
    sort_prob=np.sort(predictions[i])[::-1][:5]
    prediction_prob.append(sort_prob)

In [ ]:
prediction_index=[]
for i in range(len(predictions)):
    sort_index=np.argsort(predictions[i])[::-1][:5]
    prediction_index.append(sort_index)

In [ ]:
limit=0.7

prediction_lite=[]
for i in range(len(prediction_index)):
    if prediction_prob[i][0]<limit:
        temp=['new_individual',prediction_index[i][1],prediction_index[i][2],prediction_index[i][3],prediction_index[i][4]]
        prediction_lite.append(temp)
    elif prediction_prob[i][1]<limit:
        temp=[prediction_index[i][0],'new_individual',prediction_index[i][2],prediction_index[i][3],prediction_index[i][4]]
        prediction_lite.append(temp)
    elif prediction_prob[i][2]<limit:
        temp=[prediction_index[i][0],prediction_index[i][1],'new_individual',prediction_index[i][3],prediction_index[i][4]]
        prediction_lite.append(temp)
    elif prediction_prob[i][3]<limit:
        temp=[prediction_index[i][0],prediction_index[i][1],prediction_index[i][2],'new_individual',prediction_index[i][4]]
        prediction_lite.append(temp)
    elif prediction_prob[i][4]<limit:
        temp=[prediction_index[i][0],prediction_index[i][1],prediction_index[i][2],prediction_index[i][3],'new_individual']
        prediction_lite.append(temp)
    else:
        prediction_lite.append(prediction_index[i])

In [ ]:
#dictionary that contains info about class and class index
labels={value:key for (key,value) in train_gen.class_indices.items()}

In [ ]:
prediction_semi_final=np.empty(shape=np.array(prediction_lite).shape,dtype=np.dtype('U500'))

t=0

for i in range(len(prediction_lite)):
    for j in range(len(prediction_lite[i])):
        if prediction_lite[i][j]=='new_individual':
            prediction_semi_final[i][j]='new_individual'
        else:
            prediction_semi_final[i][j]=labels[prediction_lite[i][j]]

In [ ]:
prediction_final=np.empty(shape=np.array(prediction_lite).shape[0],dtype=np.dtype('U1000'))

for i in range(len(prediction_semi_final)):
    prediction_final[i]=" ".join([str(item) for item in prediction_semi_final[i]])


In [ ]:
#read the sample_submission csv file 
sub=pd.read_csv('/kaggle/input/happy-whale-and-dolphin/sample_submission.csv', header='infer')
sub.info()
sub["predictions"]=prediction_final
sub.to_csv('submission.csv', index=False)

In [ ]:
sub.head()